In [75]:
import sys
import os
from pathlib import Path
from PIL import Image
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Okt
from gensim.models import Word2Vec, FastText

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import optimizers, losses
from keras.layers import Dense, Dropout, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
basePath = Path("./")

imagePath = basePath/'image'
trainImagePath = imagePath/'train'
testImagePath = imagePath/'test'

testPath = basePath/'test.csv'
trainPath = basePath/'train.csv'

In [3]:
train_df = pd.read_csv(trainPath)
train_df

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [5]:
num_to_cat1 = {i:string for i,string in enumerate(train_df['cat1'].unique())}
num_to_cat2 = {i:string for i,string in enumerate(train_df['cat2'].unique())}
num_to_cat3 = {i:string for i,string in enumerate(train_df['cat3'].unique())}

cat_to_num1 = {string:i for i,string in num_to_cat1.items()}
cat_to_num2 = {string:i for i,string in num_to_cat2.items()}
cat_to_num3 = {string:i for i,string in num_to_cat3.items()}

In [6]:
train_df['category1'] = train_df['cat1'].map(cat_to_num1)
train_df['category2'] = train_df['cat2'].map(cat_to_num2)
train_df['category3'] = train_df['cat3'].map(cat_to_num3)

In [7]:
train_df

,id,img_path,overview,cat1,cat2,cat3,category1,category2,category3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,0,0,0
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,1,1,1
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,2,2,2
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,2,2,2
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,2,2,2
...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,2,2,2
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,4,7,11
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,4,7,11
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",1,1,3


In [8]:
test_df = pd.read_csv(testPath)
test_df

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [10]:
train_df[['width', 'height']] = train_df['img_path'].apply(lambda path: np.array(Image.open(path)).shape[:-1]).to_list()

In [11]:
train_df['sentence_length'] = train_df['overview'].apply(lambda sentence: len(sentence))

In [12]:
print(train_df['sentence_length'].max())
print(train_df[train_df['sentence_length'] == train_df['sentence_length'].max()]['overview'])

7972
1399    우리나라 3대 사찰의 하나로 신라 애장왕 3년(802)에 순응, 이정 두 스님이 창...
Name: overview, dtype: object


In [13]:
print(train_df['sentence_length'].min())
print(train_df[train_df['sentence_length'] == train_df['sentence_length'].min()]['overview'])

7
3030    홈페이지 참조
Name: overview, dtype: object


In [14]:
print(train_df['sentence_length'].quantile(0.01))
print(train_df[train_df['sentence_length'] == train_df['sentence_length'].quantile(0.01)]['overview'])

45.0
2854     편안한 휴식공간과 잣나무 숲으로 이루어진 피톤치드가 많은 펜션 주차 공간도 많다.
3111     직접 재배한 허브들로 맛을 낸 돈가스와 파스타,  치킨까스를 판매하는 양식당이다.
4598     조개터에 위치한 매운닭발 전문점이다. 닭발 외에도 다양한 메뉴들도 판매하고 있다.
5316     용편면에 위치한 한식당으로, 돼지갈비 및 한우 국밥을 주력 메뉴로 삼는 맛집이다.
6731     돼지갈비도 유명하다. 대표메뉴는 양념갈비다. 강원도 원주시에 있는 한식전문점이다.
6998     숯불구이 전문점이다. 대표메뉴는 돼지갈비다. 강원도 강릉시에 있는 한식전문점이다.
9013     평창군 미탄면에 위치한 곳으로 각종 과일이 들어간 새콤달콤한 막구수 전문점입니다.
9792     계절에 따라 각각의 맛이 살아있는 담백한 민물회를 만날 수 있는 곳이다. <br>
10183    바베큐 맛의 족발이다. 경기도 평택시에 있는 한식전문점이다. 대표메뉴는 족발이다.
10361    석쇠불고기, 수육, 소고기국밥, 파전 등을 전문으로 하는 한식 전문점이다.<br>
11173    대화면에 위치한 기사식당은 저렴한 가격에 다양한 반찬들로 든든한 한끼를 제공한다.
11277    은하수와여울 펜션은 조양강이 앞으로 흐르고 카페열차가 운행하며 조용하고 청결하다.
11710    대화면에 위치한 맛집으로, 추운날 먹기 좋은 감자탕을 전문으로 판매하는 식당이다.
12220    장어 및 백숙, 오리, 다슬기 해장국, 곤드레 가마솥 밥을 맛볼 수 있는 곳이다.
13473    강원도 원주시에 있는 한식당이다. 대표메뉴는 부대찌개다. 부대찌개 육수가 맛있다.
15023    한우구이 전문점이다. 대표메뉴는 등심이다. 전라남도 해남군에 있는 한식전문점이다.
15325    1999년 개설된 시장으로 전남 목포시 산정동에 위치한 상가건물형의 소형시장이다.
16030    오랜 시간 부산의 역사와 함께한 거리 남포동의 족발 맛집으로 냉채족발이 유명하다.
Name:

In [15]:
train_df

,id,img_path,overview,cat1,cat2,cat3,category1,category2,category3,width,height,sentence_length
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,0,0,0,466,699,217
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,1,1,1,483,940,244
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,2,2,2,467,700,160
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,2,2,2,525,700,67
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,2,2,2,467,700,63
...,...,...,...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,2,2,2,467,700,50
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,4,7,11,467,700,471
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,4,7,11,467,700,282
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",1,1,3,467,700,158


In [50]:
train_df.to_csv('train_processed1.csv', index=False)

In [56]:
train_df = pd.read_csv('train_processed1.csv')
train_df

,id,img_path,overview,cat1,cat2,cat3,category1,category2,category3,width,height,sentence_length
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,0,0,0,466,699,217
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,1,1,1,483,940,244
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,2,2,2,467,700,160
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,2,2,2,525,700,67
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,2,2,2,467,700,63
...,...,...,...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,2,2,2,467,700,50
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,4,7,11,467,700,471
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,4,7,11,467,700,282
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",1,1,3,467,700,158


In [52]:
min_img = Image.open(train_df[train_df['width']==train_df['width'].min()]['img_path'].values[0])
resize_img = min_img.resize((512,512))

min_img = Image.fromarray(np.uint8(min_img))
resize_img = Image.fromarray(np.uint8(resize_img))

In [53]:
max_img = Image.open(train_df[train_df['width']==train_df['width'].max()]['img_path'].values[0])
resize_img2 = max_img.resize((512,512))

max_img = Image.fromarray(np.uint8(max_img))
resize_img2 = Image.fromarray(np.uint8(resize_img2))

In [ ]:
read = tf.io.read_file('./image/train/TRAIN_16984.jpg')
array = tf.image.decode_image(read)
resize = tf.image.resize_with_crop_or_pad(array, 512, 768)
resize

In [34]:
for idx, row in train_df.iterrows():
    path = row['img_path']
    file_name = path.split('/')[-1]
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image)
    image = tf.image.resize_with_crop_or_pad(image, 512, 768)
    image = tf.image.encode_png(image)
    tf.io.write_file('./image/resized_train/'+file_name, image)

In [57]:
train_df['resized_img_path'] = './image/resized_train/' + train_df['img_path'].str.split('/').str[-1]
train_df

,id,img_path,overview,cat1,cat2,cat3,category1,category2,category3,width,height,sentence_length,resized_img_path
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,0,0,0,466,699,217,./image/resized_train/TRAIN_00000.jpg
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,1,1,1,483,940,244,./image/resized_train/TRAIN_00001.jpg
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,2,2,2,467,700,160,./image/resized_train/TRAIN_00002.jpg
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,2,2,2,525,700,67,./image/resized_train/TRAIN_00003.jpg
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,2,2,2,467,700,63,./image/resized_train/TRAIN_00004.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,2,2,2,467,700,50,./image/resized_train/TRAIN_16981.jpg
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,4,7,11,467,700,471,./image/resized_train/TRAIN_16982.jpg
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,4,7,11,467,700,282,./image/resized_train/TRAIN_16983.jpg
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",1,1,3,467,700,158,./image/resized_train/TRAIN_16984.jpg


In [59]:
train_df = train_df[['id','resized_img_path','overview','sentence_length','category1','category2','category3']]
train_df

,id,resized_img_path,overview,sentence_length,category1,category2,category3
0,TRAIN_00000,./image/resized_train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,217,0,0,0
1,TRAIN_00001,./image/resized_train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,244,1,1,1
2,TRAIN_00002,./image/resized_train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,160,2,2,2
3,TRAIN_00003,./image/resized_train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,67,2,2,2
4,TRAIN_00004,./image/resized_train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,63,2,2,2
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/resized_train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,50,2,2,2
16982,TRAIN_16982,./image/resized_train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,471,4,7,11
16983,TRAIN_16983,./image/resized_train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,282,4,7,11
16984,TRAIN_16984,./image/resized_train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,158,1,1,3


In [ ]:
train_df.to_csv('train_processed2.csv', index=False)

In [ ]:
train_df = pd.read_csv('train_processed2.csv')
train_df

In [16]:
allText = train_df['overview'].to_list()

In [77]:
okt = Okt()
tokenized_datas = []
for text in allText:
    tokenized_text = okt.morphs(text)
    tokenized_datas.append(tokenized_text)

In [85]:
model = FastText(sentences=tokenized_data)

In [86]:
model.wv.most_similar("고기")

[('말고기', 0.9568502902984619),
 ('닭고기', 0.9520527720451355),
 ('양고기', 0.9386282563209534),
 ('쇠고기', 0.9337173700332642),
 ('베고기', 0.9305434226989746),
 ('고래고기', 0.9292700886726379),
 ('고기구이', 0.9189282655715942),
 ('소고기', 0.9154700636863708),
 ('돼지고기', 0.9148669838905334),
 ('오리고기', 0.9125440120697021)]

In [87]:
model.save('fasttext.model')

In [88]:
model = Word2Vec.load('fasttext.model')

In [89]:
index_to_word = model.wv.index_to_key
index_to_word

['.',
 ',',
 '을',
 '의',
 '이',
 '에',
 '를',
 '있다',
 '은',
 '으로',
 '과',
 '가',
 '한',
 '수',
 '이다',
 '로',
 '(',
 'br',
 '는',
 '에서',
 '있는',
 ')',
 '등',
 '와',
 '들',
 '하고',
 '곳',
 '에는',
 '도',
 '인',
 '>',
 '.<',
 '할',
 '하는',
 '되어',
 '있어',
 '하여',
 '적',
 '다',
 '위치',
 '있으며',
 '한다',
 '된',
 '것',
 '마을',
 "'",
 '해',
 '체험',
 'm',
 '다양한',
 '시설',
 '그',
 '캠핑장',
 '문화',
 '함께',
 '\n',
 '2',
 '<',
 '장',
 '자리',
 '1',
 '대',
 '개',
 '위',
 '공원',
 '메뉴',
 '고',
 '즐길',
 '자연',
 '·',
 '이용',
 '층',
 '있고',
 '전',
 '계곡',
 '까지',
 '대표',
 '공간',
 '/>',
 '/><',
 '운영',
 '산',
 '여',
 '지역',
 '및',
 '3',
 '제',
 '거리',
 '또한',
 '때',
 '만',
 '주변',
 '맛',
 '부터',
 '중',
 '제공',
 '건물',
 '하였다',
 '내',
 '에서는',
 '되었다',
 '면',
 '‘',
 '동',
 '전통',
 '바다',
 '서',
 '문',
 '하며',
 '석',
 '카페',
 '약',
 '나',
 '된다',
 '따라',
 '원',
 '코스',
 '사용',
 '지',
 '위해',
 '에게',
 '조성',
 '가족',
 '사람',
 '4',
 '객실',
 '관광',
 '좋은',
 '연',
 '이며',
 '좋다',
 '전시',
 '’',
 '이름',
 '역사',
 '성',
 '),',
 '인근',
 '물',
 '특히',
 '마련',
 '시',
 '볼',
 '하기',
 '속',
 '전문점',
 '규모',
 '잘',
 '길',
 '천',
 '앞',
 '한국',
 '아

In [90]:
word_to_vector = model.wv.vectors
word_to_vector

array([[ 7.72800565e-01,  7.73393869e-01, -8.42040598e-01, ...,
        -1.74731994e+00, -1.72763944e+00, -3.00495243e+00],
       [ 7.37852871e-01, -8.67883623e-01, -1.04123592e+00, ...,
         1.16674256e+00, -5.08045077e-01, -8.25425446e-01],
       [-1.63745880e+00, -1.47277093e+00, -3.43140268e+00, ...,
         2.79060984e+00,  1.72293413e+00,  5.44081116e+00],
       ...,
       [-8.66782665e-02, -2.56479550e-02, -7.76900128e-02, ...,
        -2.94618338e-01, -1.78780988e-01, -1.29626393e-01],
       [-7.96454698e-02,  3.42799462e-02, -1.55045567e-02, ...,
        -7.57437497e-02, -5.98941259e-02,  5.47858961e-02],
       [-1.49169825e-02,  2.49713892e-03, -1.29979067e-02, ...,
        -3.51253003e-02, -5.27744144e-02, -7.28189945e-03]], dtype=float32)

In [62]:
sequences = []
for overview in tokenized_data:
    sequence = []
    for x in overview:
        try:
            sequence.append(word_to_index[x])
        except:
            sequence.append(-1)
    sequences.append(sequence)

In [66]:
train_df['sentence_length'].describe()

count    16986.000000
mean       334.291711
std        283.720345
min          7.000000
25%        168.250000
50%        274.000000
75%        415.000000
max       7972.000000
Name: sentence_length, dtype: float64

In [67]:
padded_sequences = pad_sequences(sequences, maxlen=400, padding='post')

In [69]:
padded_sequences[0]

array([   -1,     8,  1497,   220,     9,   137,    -1,   491,  2757,
           9,  5564,  1064,   476,     2,   473,   438,  3808,     2,
        3654,    40,   105,   726,    15,    28,   503,     0,   932,
          91,     5,   190,    42,   227,    -1,    24,     8,   220,
         123,    24,     3,    -1,   369,   247,     2,  2313,  1111,
        1035,  2662,    40,  1478,    89,   220,     3,  7859,     8,
         105,     3,   847,   546,     2,   395,  2988,    20,   382,
         364,     0,    88,     1,   220,    27, 16636,    82,   730,
           1,  7407,   173,    22,     3,  2922,    11,  3974,  6320,
          40,     1,   487,  5225, 13675,  5654,     4,  4434,     7,
          31,    17,    30,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [76]:
model = FastText(sentences=tokenized_data)

In [108]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data